<a href="https://colab.research.google.com/github/tomonari-masada/course2023-nlp/blob/main/13_finetuning_GPTQ_LLM_with_LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPTQで量子化されたLLMをLoRAでfinetuningする

## LLMを効率よくカスタマイズするには？
* パラメータ数が数十億（数ビリオン）のLLMをfinetuningするのは、大変。
* しかし、あらかじめ量子化されたモデルを、LoRAでfinetuningするなら、計算資源はさほど必要ない。
* ただし・・・
  * そもそも、量子化されたモデルは、元のモデルよりも性能が良くない。
* さらに・・・
  * LoRAでfinetuningするよりも、元のモデルを直接finetuningするほうが、性能は上がる。
  * しかし、元のモデルを直接finetuningするには、それなりの計算資源が必要となる。

## 参考資料
* 今回の授業資料の元になっているmediumの記事
  * https://dsmonk.medium.com/training-and-deploying-of-quantized-llms-with-lora-and-gptq-part-2-2-ec7b54659c9e

* 以下は、その他の参考資料
  * https://medium.com/@pazuzzu/in-depth-llm-fine-tuning-guide-efficiently-fine-tune-and-use-zephyr-7b-beta-assistant-using-lora-e23d8151e067
  * https://huggingface.co/docs/trl/v0.7.4/en/sft_trainer
  * https://blog.gopenai.com/fine-tuning-mistral-7b-instruct-model-in-colab-a-beginners-guide-0f7bebccf11c
  * https://github.com/huggingface/trl/blob/main/examples/scripts/sft.py

## インストール

In [ ]:
!pip install git+https://github.com/huggingface/transformers trl accelerate torch bitsandbytes peft datasets auto-gptq optimum

**ここでセッションを再起動する。**

## データセット
* 今回は、ライブドアニュースコーパスの本文部分を使ってみる。

In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "shunk031/livedoor-news-corpus",
    train_ratio=0.8,
    val_ratio=0.1,
    test_ratio=0.1,
    random_state=42,
    shuffle=True,
)

In [ ]:
dataset

In [ ]:
dataset["train"]["content"][0]

## モデル
* ELYZAをGPTQで量子化したモデルを使う。

In [ ]:
model_id = "TFMC/ELYZA-japanese-Llama-2-7b-instruct-GPTQ-4bit-64g"

* 細かいノウハウがあるようなので、今回の資料の元になっている下の記事を参照のこと。
  * https://dsmonk.medium.com/training-and-deploying-of-quantized-llms-with-lora-and-gptq-part-2-2-ec7b54659c9e

In [ ]:
import torch
from transformers import AutoTokenizer
from auto_gptq import AutoGPTQForCausalLM
from peft import prepare_model_for_kbit_training
from transformers import GPTQConfig

model = AutoGPTQForCausalLM.from_quantized(
    model_id,
    use_safetensors=True,
    disable_exllama=False,
    device="cuda:0",
    trust_remote_code=True,
    )

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model.config.use_cache = False

# https://github.com/huggingface/transformers/pull/24906
#disable tensor parallelism
model.config.pretraining_tp = 1

## LoRAの設定

* LoRAはPEFTのなかで最もよく使われている手法。
  * トランスフォーマモデルを構成する様々なパラメータ行列をfinetuningする。
  * ただし、元のモデルはfreezeさせて、finetuningによる差分だけ学習する。
  * そして、差分そのものを学習するのではなく、差分の低ランク近似を学習する。
* 詳しくは、原論文を参照。
  * https://arxiv.org/abs/2106.09685

In [ ]:
from peft import LoraConfig, get_peft_model

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["k_proj","o_proj","q_proj","v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

## finetuningの設定

* `remove_unused_columns=False`
  * これを追加しないと、なぜかデータセットに関するエラーが出てしまう。
  * https://discuss.huggingface.co/t/indexerror-invalid-key-16-is-out-of-bounds-for-size-0/14298/24

* 下記の設定は適当に決めたもの。
  * チューニングしないと性能が出ない。

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    warmup_steps=10,
    max_steps=100,
    learning_rate=2e-5,
    fp16=True, #use mixed precision training
    logging_steps=1,
    output_dir="outputs_gptq_training",
    optim="adamw_hf",
    save_strategy="epoch",
    remove_unused_columns=False,
    )

## `SFTTrainer`クラス

* TRL = Transformer Reinforcement Learning
  * https://huggingface.co/docs/trl/
  * TRLは強化学習によってLLMをfinetuningするためのライブラリだが・・・
  * 今回は、単にSFT (supervised finetuning)をおこなうために使うだけ。

In [ ]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    peft_config=config,
    dataset_text_field="content",
    tokenizer=tokenizer,
    packing=False,
    max_seq_length=512,
    )

## 学習の実行

* 20分ぐらいかかる。

In [ ]:
trainer.train()

## 評価

* 時間節約のため・・・
* validation setのインスタンスの`title`で・・・
* 最も近いインスタンスが同じ`category`である割合を調べる。

In [ ]:
tokenizer.add_eos_token = True
batch_dict = tokenizer(
    dataset["validation"]["title"][0],
    add_special_tokens=True,
    padding=True,
    return_tensors="pt",
    )

In [ ]:
batch_dict.input_ids

In [ ]:
batch_dict.attention_mask

In [ ]:
with torch.no_grad():
  last_hidden_state = model(
      input_ids=batch_dict.input_ids.to(model.device),
      attention_mask=batch_dict.attention_mask.to(model.device),
      output_hidden_states=True,
      ).hidden_states[-1].cpu()

In [ ]:
last_hidden_state.shape

In [ ]:
batch_dict.input_ids[0,-1]

In [ ]:
(batch_dict.input_ids == tokenizer.eos_token_id).nonzero()

In [ ]:
last_hidden_state[0,-1]

In [ ]:
len(dataset["validation"])

In [ ]:
from tqdm import tqdm_notebook

tokenizer.add_eos_token = True

embeddings_list = list()
for i in tqdm_notebook(range(len(dataset["validation"]))):
  batch_dict = tokenizer(
    dataset["validation"]["title"][i],
    add_special_tokens=True,
    padding=True,
    return_tensors="pt",
    )
  with torch.no_grad():
    last_hidden_state = model(
        input_ids=batch_dict.input_ids.to(model.device),
        attention_mask=batch_dict.attention_mask.to(model.device),
        output_hidden_states=True,
        ).hidden_states[-1].cpu()
    embeddings = last_hidden_state[0,-1]
    embeddings_list.append(embeddings)

In [ ]:
embeddings = torch.stack(embeddings_list).type(torch.float32)

In [ ]:
embeddings.shape

In [ ]:
import torch.nn.functional as F

similarities = torch.matmul(
    F.normalize(embeddings, dim=-1),
    F.normalize(embeddings, dim=-1).t()
)

In [ ]:
similarities

In [ ]:
sorted_indices = torch.argsort(similarities, descending=True).cpu()

In [ ]:
sorted_indices

In [ ]:
sorted_indices[:,1]

In [ ]:
validation_categories = torch.tensor(dataset["validation"]["category"], dtype=torch.int64)

In [ ]:
validation_categories[sorted_indices[:,1]]

In [ ]:
(validation_categories == validation_categories[sorted_indices[:,1]]).sum()

In [ ]:
len(validation_categories)

In [ ]:
368 / 737